# ++ Carregar dados históricos da Bovespa em Dataframe para análise ++

In [1]:
import time
time.asctime( time.localtime(time.time()) )

'Tue May 31 13:42:43 2022'

In [2]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__

'1.3.4'

In [3]:
import matplotlib as mat
mat.__version__

'3.4.3'

In [4]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [5]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [7]:
# vr_investimento_padrao = 20000

In [8]:
def carrega_dados():
    layout = [2, 8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]
#     df = pd.read_fwf("arquivos/COTAHIST_A2021.ZIP", widths=layout, compression='zip',engine='c',
#                     names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df = pd.read_fwf("arquivos/COTAHIST_A2022.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

################## temporário #####################
    df_112021 = pd.read_fwf("arquivos/COTAHIST_M112021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df_122021 = pd.read_fwf("arquivos/COTAHIST_M122021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])

    df = pd.concat([df_112021, df_122021, df])

################## temporário #####################

    df = df.loc[df["CODBDI"] == 2]
    
    df02 = pd.DataFrame({
         "cdAcao":df["CODNEG"]
        ,"dtPregao":pd.to_datetime(df["DTEXCH"], format="%Y%m%d", errors="ignore")
        ,"vrFech":df["PREULT"] / 100
        ,"vrVolume":df["VOLTOT"] / 100
        ,"vrMaxDia":df["PREMAX"] / 100
        ,"vrMinDia":df["PREMIN"] / 100
        ,"vrAbert":df["PREABE"] / 100
    })
    df02 = df02.sort_values(["cdAcao", "dtPregao"], ascending=True)
    
    df02["pcVar"], df02["pcMaxDia"], df02["pcMinDia"], df02["pcAbert"] = [
         ((df02.vrFech   / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMaxDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMinDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrAbert  / df02.vrFech.shift(1)) -1) *100
        ]
    
    df02["ic05"], df02["ic10"], df02["ic15"], df02["ic20"], df02["ic25"], df02["ic30"] = [
         df02.apply(condicao05, axis=1)
        ,df02.apply(condicao10, axis=1)
        ,df02.apply(condicao15, axis=1)
        ,df02.apply(condicao20, axis=1)
        ,df02.apply(condicao25, axis=1)
        ,df02.apply(condicao30, axis=1)
        ]

    # Filtrando Ação
    # df02.loc[df02["cdAcao"]=="AMBP3"].sort_values(["cdAcao", "dtPregao"], ascending=False)
    
    return df02

In [9]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0

In [10]:
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0

In [11]:
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0

In [12]:
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0

In [13]:
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0

In [14]:
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [15]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [16]:
df = carrega_dados()

In [17]:
# Seleciona últimos 25 dias de cada ação
df25d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[24])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df20d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[19])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df15d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[14])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df10d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[9])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [18]:
df05 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol = df25d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech = df25d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [19]:
df05_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_10d = df10d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_10d = df10d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_10d = df10d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [20]:
df05_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_15d = df15d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_15d = df15d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [21]:
df05_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_20d = df20d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_20d = df20d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [22]:
df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"]]
df05_10d["1.0%"], df05_10d["1.5%"], df05_10d["2.0%"], df05_10d["2.5%"], df05_10d["3.0%"] , df05_10d["AvgVol"], df05_10d["AvgVrFech"] = [
    df10_10d["resultado"],df15_10d["resultado"],df20_10d["resultado"],df25_10d["resultado"],df30_10d["resultado"],dfVol_10d["vol"],dfVrFech_10d["vrFech"]]
df05_15d["1.0%"], df05_15d["1.5%"], df05_15d["2.0%"], df05_15d["2.5%"], df05_15d["3.0%"] , df05_15d["AvgVol"], df05_15d["AvgVrFech"] = [
    df10_15d["resultado"],df15_15d["resultado"],df20_15d["resultado"],df25_15d["resultado"],df30_15d["resultado"],dfVol_15d["vol"],dfVrFech_15d["vrFech"]]
df05_20d["1.0%"], df05_20d["1.5%"], df05_20d["2.0%"], df05_20d["2.5%"], df05_20d["3.0%"] , df05_20d["AvgVol"], df05_20d["AvgVrFech"] = [
    df10_20d["resultado"],df15_20d["resultado"],df20_20d["resultado"],df25_20d["resultado"],df30_20d["resultado"],dfVol_20d["vol"],dfVrFech_20d["vrFech"]]

In [23]:
df05     =     df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_10d = df05_10d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_15d = df05_15d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_20d = df05_20d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

In [24]:
pd.set_option('display.max_rows', None)
# df06 = df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 20) & (df05["AvgVrFech"] > 5)]

In [25]:
df25d['vrVolume'] = df25d['vrVolume'].astype(int)
df25d['vrVolume'] = df25d['vrVolume'].map('{:,}'.format)

### 25 dias, order by 1%

In [26]:
df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 22) & (df05["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
1151,TRAD3,23,23,22,20,18,17,"16,137,293.48",5.11
1001,RECV3,23,22,20,19,19,16,"30,545,361.68",26.43
955,PRIO3,23,22,19,16,15,11,"517,279,005.52",26.29
87,ANIM3,23,22,18,16,13,10,"15,098,992.60",5.40


### 20 dias, order by 1%

In [39]:
df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["1.0%"] >= 18) & (df05_20d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
1134,TRAD3,18,18,17,15,13,13,"14,430,406.70",5.11
989,RECV3,19,18,16,15,15,13,"32,976,983.65",26.91
945,PRIO3,19,18,16,13,12,8,"458,242,324.90",26.42


### 15 dias, order by 1%

In [38]:
df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["1.0%"] >= 15) & (df05_15d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
963,RECV3,15,15,13,12,12,12,"33,163,570.40",27.52
1036,SLCE3,15,15,11,8,8,4,"98,923,958.07",53.37


### 10 dias, order by 1%

In [37]:
df05_10d.loc[(df05_10d["AvgVol"] > 6000000) & (df05_10d["1.0%"] >= 10) & (df05_10d["AvgVrFech"] > 5)]

,cdAcao,0.5%,1.0%,1.5%,2.0%,2.5%,3.0%,AvgVol,AvgVrFech
916,RECV3,10,10,9,8,8,8,"37,811,355.40",28.29
996,SQIA3,10,10,9,7,6,6,"18,665,986.60",19.24
926,RRRP3,10,10,9,7,6,5,"128,195,457.20",45.09
1049,TOTS3,10,10,9,7,3,2,"152,376,616.30",26.97
908,RAIZ4,10,10,8,8,7,7,"84,117,525.50",5.83
875,PRIO3,10,10,8,7,6,5,"502,204,995.40",26.78
887,QUAL3,10,10,7,6,4,2,"28,901,102.10",11.62
985,SLCE3,10,10,7,4,4,2,"92,058,127.10",53.84
63,AGRO3,10,10,6,4,3,3,"19,005,987.70",31.22


In [30]:
### 25 dias, Order by 0.5% # df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5% # df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5% # df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

### Consulta ação específica

In [46]:
df25d.loc[(df25d["cdAcao"] == "RECV3")].replace(0, "")

,cdAcao,dtPregao,vrFech,vrVolume,vrMaxDia,vrMinDia,vrAbert,pcVar,pcMaxDia,pcMinDia,pcAbert,ic05,ic10,ic15,ic20,ic25,ic30
168794,RECV3,2022-05-30,29.65,"21,033,321",30.90,29.56,30.15,-1.17,3.00,-1.47,0.50,1,1,1,1,1,1
167276,RECV3,2022-05-27,30.00,"33,512,664",30.13,28.91,29.09,3.23,3.68,-0.52,0.10,1,1,1,1,1,1
165590,RECV3,2022-05-26,29.06,"35,653,314",30.48,29.06,29.38,-0.89,3.96,-0.89,0.20,1,1,1,1,1,1
163919,RECV3,2022-05-25,29.32,"42,309,445",29.54,27.93,27.97,4.34,5.12,-0.60,-0.46,1,1,1,1,1,1
162277,RECV3,2022-05-24,28.10,"26,879,814",28.56,27.51,27.63,0.25,1.89,-1.86,-1.43,1,1,1,,,
160600,RECV3,2022-05-23,28.03,"45,316,768",29.24,28.03,28.66,-1.16,3.10,-1.16,1.06,1,1,1,1,1,1
158898,RECV3,2022-05-20,28.36,"48,791,674",28.36,27.39,27.78,3.13,3.13,-0.40,1.02,1,1,1,1,1,1
157179,RECV3,2022-05-19,27.50,"52,097,285",27.60,25.56,25.70,6.59,6.98,-0.93,-0.39,1,1,1,1,1,1
155513,RECV3,2022-05-18,25.80,"32,318,485",27.42,25.74,27.24,-4.76,1.22,-4.98,0.55,1,1,,,,
153818,RECV3,2022-05-17,27.09,"40,200,784",27.43,26.82,27.00,2.27,3.55,1.25,1.93,1,1,1,1,1,1


## Definindo valores para compra

In [32]:
# Valor disponível para cmp / 1000
# Valor da ação no leilão
vr_corretagem = 9.8
emolumentos = 0.000325
acao = 'LIGT3'
vr_disp = 110
vr_acao_leilao = 1148 / 100
vr_qtd = round((vr_disp * 1000) / (vr_acao_leilao * 100)) * 100

In [33]:
vr_cmp = round(vr_acao_leilao * 1.014, 2)

vr_tot_cmp = vr_acao_leilao * vr_qtd
corret_cmp = vr_corretagem + (vr_tot_cmp * emolumentos)
vr_tot_liq_cmp = vr_tot_cmp + corret_cmp

vr_acao_vnd_10 = float('{0:.2f}'.format(vr_acao_leilao * 1.0108 + 0.01))
vr_tot_vnd_10 = vr_acao_vnd_10 * vr_qtd
corret_vnd_10 = vr_corretagem + (vr_tot_vnd_10 * emolumentos)
vr_tot_liq_vnd_10 = vr_tot_vnd_10 - corret_vnd_10

vr_acao_vnd_15 = float('{0:.2f}'.format(vr_acao_leilao * 1.0158 + 0.01))
vr_tot_vnd_15 = vr_acao_vnd_15 * vr_qtd
corret_vnd_15 = vr_corretagem + (vr_tot_vnd_15 * emolumentos)
vr_tot_liq_vnd_15 = vr_tot_vnd_15 - corret_vnd_15

In [34]:
d = {'Compra':     ['Quantidade', 'Valor Leilão', 'Total', 'Total líquido', 'Corretagem Compra'],
     ' ':          [vr_qtd, vr_acao_leilao, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
     'Venda 1,0%': ['Valor compra', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,0%':       [vr_cmp, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
     'Venda 1,5%': [' ', 'Valor', 'Total', 'Total líquido', 'Corretagem Venda'],
     '1,5%':       [' ', vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
    }
df_calc = pd.DataFrame(data=d)
df_calc

,Compra,,"Venda 1,0%","1,0%","Venda 1,5%","1,5%"
0,Quantidade,"9,600.00",Valor compra,11.64,,
1,Valor Leilão,11.48,Valor,11.61,Valor,11.67
2,Total,"110,208.00",Total,"111,456.00",Total,"112,032.00"
3,Total líquido,"110,253.62",Total líquido,"111,409.98",Total líquido,"111,985.79"
4,Corretagem Compra,45.62,Corretagem Venda,46.02,Corretagem Venda,46.21


In [35]:
d2 = {'Index':      ['Quantidade', 'Valor', 'Total', 'Total líquido', 'Corretagem'],
      'Compra':     [vr_qtd, vr_cmp, vr_tot_cmp, vr_tot_liq_cmp, corret_cmp],
      'Venda 1.0%': [vr_qtd, vr_acao_vnd_10, vr_tot_vnd_10, vr_tot_liq_vnd_10, corret_vnd_10],
      'Venda 1.5%': [vr_qtd, vr_acao_vnd_15, vr_tot_vnd_15, vr_tot_liq_vnd_15, corret_vnd_15]
     }
df_calc2 = pd.DataFrame(data=d2)
df_calc2.set_index('Index')
df_calc2

,Index,Compra,Venda 1.0%,Venda 1.5%
0,Quantidade,"9,600.00","9,600.00","9,600.00"
1,Valor,11.64,11.61,11.67
2,Total,"110,208.00","111,456.00","112,032.00"
3,Total líquido,"110,253.62","111,409.98","111,985.79"
4,Corretagem,45.62,46.02,46.21


In [36]:
time.asctime( time.localtime(time.time()) )

'Tue May 31 13:43:47 2022'